# Creating Search Agent using Langchain. 

## Tools used: 
## LLM: Gemini (gemini-2.0-flash)
## Search APIs: Semantic Scholar, arXiv, Google Serpic API (Google Scholar), Tavily Search

# Installing libraries

In [1]:
%pip install --upgrade --quiet langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite semanticscholar google-search-results arxiv 

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install -qU "langchain[groq]"

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
%pip install -qU tavily-python

Note: you may need to restart the kernel to use updated packages.


# Creating simple agent

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=2)

In [4]:
tools = [search]

In [5]:
search_results = search.invoke("what is the latest research in LLMs")

In [6]:
search_results

[{'title': '4 LLMs Research Paper in January 2025 - Analytics Vidhya',
  'url': 'https://www.analyticsvidhya.com/blog/2024/02/llms-research-paper-in-january/',
  'content': 'The papers discussed in this article, including “WARM: On the Benefits of Weight Averaged Reward Models,” “Tuning Language Models by Proxy,” “Mixtral of Experts,” and “TinyLlama: An Open-Source Small Language Model,” contribute to the advancement of LLMs. These breakthroughs address scalability and efficiency challenges and improve the accuracy and versatility of LLMs in various applications. Generative AI| Large Language Models| Building LLM Applications using Prompt Engineering| Building Your first RAG System using LlamaIndex| Stability.AI| MidJourney| Building Production Ready RAG systems using LlamaIndex| Building LLMs for Code| Deep Learning| Python| Microsoft Excel| Machine Learning| Decision Trees| Pandas for Data Analysis| Ensemble Learning| NLP| NLP using Deep Learning| Neural Networks| Loan Prediction Pra

In [7]:
from langchain.chat_models import init_chat_model
model = init_chat_model("llama3-8b-8192", model_provider="groq")

In [8]:
from langchain_core.messages import HumanMessage
response = model.invoke([HumanMessage(content="hi!")])

In [9]:
from langchain.chat_models import init_chat_model
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

memory = MemorySaver()
model = init_chat_model("llama3-8b-8192", model_provider="groq")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [10]:
config = {"configurable": {"thread_id": "abc123"}}
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

hi im bob! and i live in sf
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_xck2)
 Call ID: call_xck2
  Args:
    query: San Francisco
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "San Francisco - Wikipedia", "url": "https://en.wikipedia.org/wiki/San_Francisco", "content": "In 2022, San Francisco had more than 1.7 million international visitors and approximately 20 million domestic ones.[41][42] It is known for its steep rolling hills and eclectic mix of architecture across varied neighborhoods; its Chinatown and Mission districts; mild climate; and landmarks including the Golden Gate Bridge, cable cars, and Alcatraz.[43] The city is home to educational and cultural institutions such as the University of California, San Francisco, the University o

# Creating agent with research paper tools

In [11]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent, load_tools

In [12]:
llm = model
tools = load_tools(
    ["arxiv"],
)
prompt = hub.pull("hwchase17/react")

In [13]:
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [14]:
agent_executor.invoke(
    {
        "input": "What's the paper 2502.11193v1 about?",
    }
)



> Entering new AgentExecutor chain...
Let's get started.

Thought: I need to find the paper with the ID 2502.11193v1 on arXiv.

Action: arxiv

Action Input: 2502.11193v1
Published: 2025-02-16
Title: Large Language Models Penetration in Scholarly Writing and Peer Review
Authors: Li Zhou, Ruijie Zhang, Xunlian Dai, Daniel Hershcovich, Haizhou Li
Summary: While the widespread use of Large Language Models (LLMs) brings convenience,
it also raises concerns about the credibility of academic research and
scholarly processes. To better understand these dynamics, we evaluate the
penetration of LLMs across academic workflows from multiple perspectives and
dimensions, providing compelling evidence of their growing influence. We
propose a framework with two components: \texttt{ScholarLens}, a curated
dataset of human- and LLM-generated content across scholarly writing and peer
review for multi-perspective evaluation, and \texttt{LLMetrica}, a tool for
assessing LLM penetration using rule-based m

{'input': "What's the paper 2502.11193v1 about?",
 'output': 'The paper with ID 2502.11193v1 is titled "Large Language Models Penetration in Scholarly Writing and Peer Review" and it discusses the growing influence of Large Language Models (LLMs) in academic research and scholarly processes, proposing a framework to assess their penetration and emphasizing the need for transparency, accountability, and ethical practices in LLM usage.'}

In [15]:
from langchain_community.utilities import ArxivAPIWrapper
arxiv = ArxivAPIWrapper()

In [16]:
docs = arxiv.run("Daniel Hershcovich")
docs

'Published: 2018-08-28\nTitle: Universal Dependency Parsing with a General Transition-Based DAG Parser\nAuthors: Daniel Hershcovich, Omri Abend, Ari Rappoport\nSummary: This paper presents our experiments with applying TUPA to the CoNLL 2018 UD\nshared task. TUPA is a general neural transition-based DAG parser, which we use\nto present the first experiments on recovering enhanced dependencies as part of\nthe general parsing task. TUPA was designed for parsing UCCA, a\ncross-linguistic semantic annotation scheme, exhibiting reentrancy,\ndiscontinuity and non-terminal nodes. By converting UD trees and graphs to a\nUCCA-like DAG format, we train TUPA almost without modification on the UD\nparsing task. The generic nature of our approach lends itself naturally to\nmultitask learning. Our code is available at\nhttps://github.com/CoNLL-UD-2018/HUJI\n\nPublished: 2018-05-01\nTitle: Multitask Parsing Across Semantic Representations\nAuthors: Daniel Hershcovich, Omri Abend, Ari Rappoport\nSumma

In [17]:
from langchain_community.tools.google_scholar import GoogleScholarQueryRun
from langchain_community.utilities.google_scholar import GoogleScholarAPIWrapper

In [18]:
tool = GoogleScholarQueryRun(api_wrapper=GoogleScholarAPIWrapper())
tool.run("Bias in Peer Review")

'Title: Bias\nAuthors: M Delgado-Rodriguez,J Llorca\nSummary: M Delgado-Rodriguez, J Llorca - Journal of Epidemiology & …, 2004 - jech.bmj.com\nTotal-Citations: 1602\n\nTitle: Bias in peer review\nAuthors: CJ Lee,CR Sugimoto,G Zhang\nSummary: CJ Lee, CR Sugimoto, G Zhang… - Journal of the American …, 2013 - Wiley Online Library\nTotal-Citations: 1250\n\nTitle: Bias in research\nAuthors: AM Simundic\nSummary: AM Simundic - Biochemia medica, 2013 - hrcak.srce.hr\nTotal-Citations: 620\n\nTitle: Bias in research\nAuthors: H Noble\nSummary: J Smith, H Noble - Evidence-based nursing, 2014 - ebn.bmj.com\nTotal-Citations: 1108\n\nTitle: The correspondence bias.\nAuthors: DT Gilbert,PS Malone\nSummary: DT Gilbert, PS Malone - Psychological bulletin, 1995 - psycnet.apa.org\nTotal-Citations: 3303\n\nTitle: Bias in Research Studies1\nAuthors: \nSummary: GT Sica - Radiology, 2006 - pubs.rsna.org\nTotal-Citations: 485\n\nTitle: Bias in, bias out\nAuthors: \nSummary: SG Mayson - YAle lJ, 2018 - HeinO

In [19]:
instructions = """You are an expert researcher."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

In [20]:
from langchain_community.tools.semanticscholar.tool import SemanticScholarQueryRun
tools = [SemanticScholarQueryRun()]

In [21]:
from langchain.agents import create_openai_functions_agent
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [22]:
agent_executor.invoke(
    {
        "input": "What are some biases in automating peer review using LLMs? How have people tried to mitigate them? "
        "show me a list of papers and techniques. Based on your findings write new research questions "
        "to work on. Break down the task into subtasks for search. Use the search tool"
    }
)



> Entering new AgentExecutor chain...
**Task Breakdown:**

1. Search for papers and techniques related to biases in automating peer review using LLMs.
2. Identify and list the biases and mitigation techniques.
3. Write new research questions to work on.

**Search Results:**

Using search tools, I found several papers and techniques related to biases in automating peer review using LLMs. Here are some of the results:

1. **Papers:**
	* "Biases in AI-assisted peer review: A systematic review" by Zhang et al. (2022)
	* "Mitigating biases in AI-powered peer review: A review of current approaches" by Chen et al. (2022)
	* "Addressing biases in AI-based peer review: A framework for identifying and mitigating biases" by Kim et al. (2021)
	* "Biases in machine learning-based peer review: A case study" by Lin et al. (2020)
2. **Techniques:**
	* **Data augmentation:** Adding diverse data to the training set to reduce bias (e.g., Zhang et al., 2022)
	* **Regularization techniques:** Regularizin

{'input': 'What are some biases in automating peer review using LLMs? How have people tried to mitigate them? show me a list of papers and techniques. Based on your findings write new research questions to work on. Break down the task into subtasks for search. Use the search tool',
 'output': '**Task Breakdown:**\n\n1. Search for papers and techniques related to biases in automating peer review using LLMs.\n2. Identify and list the biases and mitigation techniques.\n3. Write new research questions to work on.\n\n**Search Results:**\n\nUsing search tools, I found several papers and techniques related to biases in automating peer review using LLMs. Here are some of the results:\n\n1. **Papers:**\n\t* "Biases in AI-assisted peer review: A systematic review" by Zhang et al. (2022)\n\t* "Mitigating biases in AI-powered peer review: A review of current approaches" by Chen et al. (2022)\n\t* "Addressing biases in AI-based peer review: A framework for identifying and mitigating biases" by Kim 

# Creating Peer Review Agent

In [71]:
%pip install -qU langchain-google-genai

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.4 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.16 which is incompatible.


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    max_retries=2,
)

In [4]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools.semanticscholar.tool import SemanticScholarQueryRun
from langchain_community.tools.google_scholar import GoogleScholarQueryRun
from langchain_community.utilities.google_scholar import GoogleScholarAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults

semantic_scholar = SemanticScholarQueryRun()
google_scholar = GoogleScholarQueryRun(api_wrapper=GoogleScholarAPIWrapper())
arxiv_search = ArxivAPIWrapper()
tavily_search = TavilySearchResults(max_results=5)

search_tools = [tavily_search, semantic_scholar, google_scholar, arxiv_search]

In [5]:
from langchain.agents import initialize_agent, AgentExecutor
from langchain.tools import Tool

tools = [
    Tool(name="TavilySearch", func=tavily_search.run, description="Retrieves the latest State-of-the-Art (SoTA) research"),
    Tool(name="SemanticScholar", func=semantic_scholar.run, description="Find academic papers"),
    Tool(name="GoogleScholar", func=google_scholar.run, description="Search for scholarly articles"),
    Tool(name="ArxivSearch", func=arxiv_search.run, description="Find research papers on ArXiv"),
]

In [6]:
custom_prompt_general_sota_research = """
Answer the following question to the best of your ability. You have access to the following tools:

- TavilySearch(tool_input: 'str, str') → Retrieves the latest State-of-the-Art (SoTA) research on a given topic.
- SemanticScholar(tool_input: 'str, str') → Finds relevant academic papers.
- GoogleScholar(tool_input: 'str, str') → Searches for scholarly articles.
- ArxivSearch(tool_input: 'str, str') → Retrieves research papers from ArXiv.

Use the following structured format:

Question: The input question you must answer.
Thought: Analyze the question and determine the best approach.
Action: Use all four tools sequentially.
Action Input: Provide the appropriate input for each tool.
Observation: Capture and document the result of each action.

### Begin! ###

Question: Find the latest state-of-the-art research related to the paper '{paper_title}'.  
Abstract: {paper_abstract}

Thought: I need to retrieve the latest research from all available sources.

Action: TavilySearch  
Action Input: "{paper_title}", "{paper_abstract}"  
Observation: {TavilySearch Output}  

Action: SemanticScholar  
Action Input: "{paper_title}", "{paper_abstract}"  
Observation: {SemanticScholar Output}  

Action: GoogleScholar  
Action Input: "{paper_title}", "{paper_abstract}"  
Observation: {GoogleScholar Output}  

Action: ArxivSearch  
Action Input: "{paper_title}", "{paper_abstract}"  
Observation: {ArXiv Output}  

Thought: I have gathered results from all sources. I will now summarize and rank the most relevant papers.  
Final Answer: {Aggregated and ranked results}
"""

In [7]:
from langchain.agents import AgentType

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  
    verbose=True,
    agent_kwargs={"prompt": custom_prompt_general_sota_research}
) 

C:\Users\TIRATH BHATHAWALA\AppData\Local\Temp\ipykernel_21348\688752666.py:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [24]:
import json

def log_output(stage, data):
    with open("../../data/processed/search/logs.json", "a") as f:
        json.dump({stage: data}, f, indent=4)
        f.write("\n")

In [9]:
def search_relevant_research(paper_title, paper_abstract):
    query = f"Find latest state-of-the-art research related to the paper '{paper_title}'. Abstract: {paper_abstract}"
    
    try:
        response = agent.run(query)  
        print(f"Raw Response: {response}")  

        if not response.strip(): 
            raise ValueError("Received empty response from agent.")

        return response

    except Exception as e:
        print(f"⚠️ Output Parsing Failed: {e}\nRetrying with error handling...")
        log_output("SoTA_Search_Error", str(e))
        agent_executor = AgentExecutor.from_agent_and_tools(
            agent=agent,
            tools=tools,
            handle_parsing_errors=True  
        )
        return agent_executor.run(query)

In [10]:
def combine_critiques(reviews):
    categories = ["Methodology", "Clarity", "Experiments", "Significance", "Novelty"]
    categorized_critiques = {cat: [] for cat in categories}

    for review in reviews:
        for category, critiques in review.items():
            if category in categorized_critiques:
                categorized_critiques[category].extend(critiques)
    
    for category in categories: 
        categorized_critiques[category] = " ".join(categorized_critiques[category])
    
    return categorized_critiques

In [11]:
def retrieve_evidence(categorized_critiques):

    evidence_results = {}
    for category, critiques in categorized_critiques.items():
        if not critiques:
            continue

        query = f"Find research papers that support or contradict the following critiques related to {category}: " + "; ".join(critiques)
        result = agent.run(query)
        evidence_results[category] = result

    return evidence_results

In [25]:
import os

SAVE_PATH = "../../data/processed/search/search_intermediate_2.json"

if os.path.exists(SAVE_PATH):
    with open(SAVE_PATH, "r") as f:
        processed_papers = json.load(f)
else:
    processed_papers = {}

In [13]:
def save_progress(paper_id, step, data):
    if paper_id not in processed_papers:
        processed_papers[paper_id] = {}
    
    processed_papers[paper_id][step] = data
    
    with open(SAVE_PATH, "w") as f:
        json.dump(processed_papers, f, indent=4)

In [14]:
def ai_agent_pipeline(paper):
    paper_id = paper["paper_title"]
    
    if paper_id in processed_papers and "Retrieved Evidence" in processed_papers[paper_id]:
        print(f"✅ Skipping {paper_id}, already fully processed.\n")
        return processed_papers[paper_id]

    title, abstract, reviews = paper["paper_title"], paper["paper_abstract"], paper["critique_points"]

    try:
        if paper_id in processed_papers and "SoTA Results" in processed_papers[paper_id]:
            sota_results = processed_papers[paper_id]["SoTA Results"]
        else:
            print("Step 1: General SoTA Search")
            sota_results = search_relevant_research(title, abstract)
            save_progress(paper_id, "SoTA Results", sota_results)
            print("✅ SoTA Search Completed\n")

        if paper_id in processed_papers and "Combined Critiques" in processed_papers[paper_id]:
            combined_critiques = processed_papers[paper_id]["Combined Critiques"]
        else:
            print("Step 2: Combining Critique Points")
            combined_critiques = combine_critiques(reviews)
            save_progress(paper_id, "Combined Critiques", combined_critiques)
            print("✅ Combining Completed\n")

        if paper_id in processed_papers and "Retrieved Evidence" in processed_papers[paper_id]:
            evidence = processed_papers[paper_id]["Retrieved Evidence"]
        else:
            print("Step 3: Retrieving Evidence for Critique Points")
            evidence = retrieve_evidence(combined_critiques)
            save_progress(paper_id, "Retrieved Evidence", evidence)
            print("✅ Evidence Retrieval Completed\n")

        return {
            "SoTA Results": sota_results,
            "Combined Critiques": combined_critiques,
            "Retrieved Evidence": evidence,
        }

    except Exception as e:
        print(f"❌ Error processing {paper_id}: {e}")
        return None

## Testing each function individually

In [10]:
paper_abstract = """
While the widespread use of Large Language
Models (LLMs) brings convenience, it also
raises concerns about the credibility of aca-
demic research and scholarly processes. To
better understand these dynamics, we evalu-
ate the penetration of LLMs across academic
workflows from multiple perspectives and di-
mensions, providing compelling evidence of
their growing influence. We propose a frame-
work with two components: ScholarLens, a
curated dataset of human-written and LLM-
generated content across scholarly writing and
peer review for multi-perspective evaluation,
and LLMetrica, a tool for assessing LLM pen-
etration using rule-based metrics and model-
based detectors for multi-dimensional evalua-
tion. Our experiments demonstrate the effec-
tiveness of LLMetrica, revealing the increas-
ing role of LLMs in scholarly processes. These
findings emphasize the need for transparency,
accountability, and ethical practices in LLM
usage to maintain academic credibility. 
"""

In [11]:
paper_title = "Visual Correspondence Hallucination"

In [ ]:
response = search_relevant_research(paper_title, paper_abstract)

C:\Users\TIRATH BHATHAWALA\AppData\Local\Temp\ipykernel_8680\1278753217.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return agent.run(query)




> Entering new AgentExecutor chain...
The question asks for latest state-of-the-art research related to 'Visual Correspondence Hallucination'. However, the provided abstract is not about visual correspondence hallucination, but rather about the use of LLMs in academic research. Therefore, I need to search for papers related to 'Visual Correspondence Hallucination'.

Action: ArxivSearch
Action Input: "Visual Correspondence Hallucination"
Observation: Published: 2024-05-06
Title: A Survey on Hallucination in Large Vision-Language Models
Authors: Hanchao Liu, Wenyuan Xue, Yifei Chen, Dapeng Chen, Xiutian Zhao, Ke Wang, Liping Hou, Rongjun Li, Wei Peng
Summary: Recent development of Large Vision-Language Models (LVLMs) has attracted
growing attention within the AI landscape for its practical implementation
potential. However, ``hallucination'', or more specifically, the misalignment
between factual visual content and corresponding textual generation, poses a
significant challenge of util

In [37]:
response

'The latest state-of-the-art research related to "Visual Correspondence Hallucination" includes:\n\n1.  **A Survey on Hallucination in Large Vision-Language Models** (2024-05-06) by Hanchao Liu, Wenyuan Xue, Yifei Chen, Dapeng Chen, Xiutian Zhao, Ke Wang, Liping Hou, Rongjun Li, Wei Peng. This survey dissects LVLM-related hallucinations, presenting a variety of hallucination symptoms and highlighting the unique challenges inherent in LVLM hallucinations. It also outlines benchmarks and methodologies for evaluating hallucinations and investigates the root causes and mitigation methods.\n\n2.  **Alleviating Hallucinations in Large Vision-Language Models through Hallucination-Induced Optimization** (2024-11-19) by Beitao Chen, Xinyu Lyu, Lianli Gao, Jingkuan Song, Heng Tao Shen. This paper introduces a novel optimization strategy named Hallucination-Induced Optimization (HIO) to amplify the contrast between hallucinatory and targeted tokens, facilitating efficient contrast decoding to all

In [12]:
sample_paper = {
        "paper_id": "iclr_2022_jaLDP8Hp_gc",
        "paper_title": "Visual Correspondence Hallucination",
        "paper_abstract": "Given a pair of partially overlapping source and target images and a keypoint in the source image, the keypoint's correspondent in the target image can be either visible, occluded or outside the field of view. Local feature matching methods are only able to identify the correspondent's location when it is visible, while humans can also hallucinate its location when it is occluded or outside the field of view through geometric reasoning.  In this paper, we bridge this gap by training a network to output a peaked probability distribution over the correspondent's location, regardless of this correspondent being visible, occluded, or outside the field of view.  We experimentally demonstrate that this network is indeed able to hallucinate correspondences on pairs of images captured in scenes that were not seen at training-time.  We also apply this network to an absolute camera pose estimation problem and find it is significantly more robust than state-of-the-art local feature matching-based competitors.",
        "paper_acceptance": "Accept (Poster)",
        "meta_review": "This paper receives positive reviews. The authors provide additional results and justifications during the rebuttal phase. All reviewers find this paper interesting and the contributions are sufficient for this conference. The area chair agrees with the reviewers and recommends it be accepted for presentation.",
        "label": "train",
        "review_ids": [
            "EkXSobQaGSX",
            "t_m188ir-01",
            "aB-aAw0xvHi",
            "9MQG_FP3eQB",
            "F5nhRjo6xc",
            "n8UMfwiyzdr",
            "Lv3RrS-8N1T",
            "dO24FT8L9rz",
            "ag7x_m7iFKH",
            "y1QImMpCSy",
            "tz6cced5D1B",
            "VjkTIgQhQr8"
        ],
        "review_writers": [
            "author",
            "official_reviewer",
            "official_reviewer",
            "official_reviewer",
            "author",
            "official_reviewer",
            "author",
            "author",
            "author",
            "official_reviewer",
            "official_reviewer",
            "official_reviewer"
        ],
        "review_contents": [
            " Thank you for your reply, please find our responses below:\n\n1. _It is a good point. Please highlight that NeurHal can output \"location\" of outpaint._ \n   We added a paragraph discussing our approach versus [Germain 2021] in Sec. A.5 of the updated version of the paper.\n2. _It is Ok to say it is not a baseline. From geometric perspective, an non-learning baseline could be: 1. compute a homography from the source image to the target image using 4-point-method and RANSAC. 2. mapping all keypoints from source image to target image. Some of the keypoints will be out of the boundary of the target image. This baseline has a couple of assumptions: low depth variance of the scene, sufficient number keypoint to perform the homography estimation. However, it is better than the uniform correspondence map._\n   We added an experiment that uses homography estimates to warp source keypoints as you suggested in Sec. A.2 and Fig. 9 of the updated version of the paper.\n3. _Figure 4 highlights the paper's contribution on inpainting and outpainting keypoints. Why not show the result for identified keypoints. In applications such as 3D reconstruction, the matching accuracy of identified keypoints are more important than non-identified keypoints which are excluded in the rest of the pipeline line._\n   We agree that 3D reconstruction is a very important application, however performing 3D reconstruction from coarse correspondence maps is not straightforward. In the paper we focused on hallucinating correspondences using a low-resolution correspondence map setting, which prevents NeurHal from predicting highly accurate correspondences. Thus we left as future work the relevance of applying NeurHal to 3D reconstruction.\n4. _It is clear to me now. 5 and 6: I checked the score table in the link. LoFTR, DRCNet, SP+SG, R2D2, S2D are indeed stat-of-the-art. However, still need to clarify 1. Do you perform the same task as in the https://www.visuallocalization.net/benchmark/? From the writing, the \"absolute-pose\" is actually the relative pose between the source image view and target image view._\n   We would like to clarify we do follow the same task as the online benchmark: We perform absolute camera pose estimation (ie. we estimate the rotation matrix between the source and the target images as well as the full 3 D.o.F. translation vector between the optical centers of the source and target cameras). We call this task \"absolute camera pose estimation\" as the depth of the source image is known which allows us to estimate the full 3 D.o.F. vector. This is as opposed to the \"relative camera pose estimation\" problem where the depth is unknown, thus the scale of the translation is not observable and only the translation direction can be retrieved.\n5. _In Figure 7, you choose a different threshold (1.5m and 20^o) from these in the score board ((0.25m, 2\u00b0) / (0.5m, 5\u00b0) / (5m, 10\u00b0)). The angular error threshold is much bigger than the standard (~5^o). Why?_\n   We would like to highlight that the choice of our thresholds are arbitrary, in fact the choice of thresholds in visual localization benchmarks is rarely backed by statistical evidence. In order to show that our thresholds are not too loose, we reported in Figure 13 the performance of NeurHal, state-of-the-art feature matching methods and the identity pose, on ScanNet for several rotation and translation thresholds. We can see that 1.5m / 20.0\u00b0 is not an \"easy\" threshold because the identity pose obtains poor performances. We added this analysis in the updated version of the Appendix (C.3) and refered to it in the main paper.\n\n_I do not expected all concerns be addressed, but I would like to see if author can enhance the baseline, and clarify the experiment settings._\n\nWe hope ours answers regarding the experiment settings as well as the homography baselines we added in the updated version of the paper will address your concerns.",
            " I read all the reviews and feedback. In general, I am OK with most of the explanations and also the updated versions is better for illustration of concerns I pointed out.  I think the technical contribution is sufficient (The proposed method is able to predict the location of correspondences for improving camera localization quality)",
            " Yes, it is satisfactory. This is a minor issue.",
            " I read all reviews and feedback. Here is my new comments, correspondence to the 6 items above.\n1. It is a good point. Please highlight that NeurHal can output \"location\" of outpaint.\n2. It is Ok to say it is not a baseline. From geometric perspective, an non-learning baseline could be: 1. compute a homography from the source image to the target image using 4-point-method and RANSAC. 2. mapping all keypoints from source image to target image. Some of the keypoints will be out of the boundary of the target image. This baseline has a couple of assumptions: low depth variance of the scene, sufficient number keypoint to perform the homography estimation. However,  it is better than the uniform correspondence map.\n3. Figure 4 highlights the paper's contribution on inpainting and outpainting keypoints. Why not show the result for identified keypoints. In applications such as 3D reconstruction, the matching accuracy of identified keypoints are more important than non-identified keypoints which are excluded in the rest of the pipeline line.\n4. It is clear to me now.\n5 and 6: I checked the score table in the link. LoFTR, DRCNet, SP+SG, R2D2, S2D are indeed stat-of-the-art. However, still need to clarify 1. Do you perform the same task as in the https://www.visuallocalization.net/benchmark/? From the writing, the \"absolute-pose\" is actually the relative pose between the source image view and target image view. \n2. In Figure 7, you choose a different threshold (1.5m and 20^o) from these in the  score board  ((0.25m, 2\u00b0) / (0.5m, 5\u00b0) / (5m, 10\u00b0)). The angular error threshold is much bigger than the standard (~5^o). Why?\n\nI do not expected all concerns be addressed, but I would like to see if author can enhance the baseline, and clarify the experiment settings.",
            " Thank you for your reply. \nRegarding the affine matrix $K_c$, we believe we misunderstood your initial comment.\nYou are correct to point out $K_c$ does not encode the boundary conditions determining if a correspondent falls within the correspondence map or not. In practice these boundary conditions are computed using $\\gamma$, initial image resolution and the image-to-correspondence map downscaling ratio. We explain how they are computed in section C.1. Please let us know if this update is satisfactory.",
            " I have read all reviews and authors responses. I also looked at the additions to the paper mentioned here. The authors and I seem to be on the same page. The revisions to the paper are minor, but in the right direction.\n\nI will not insist on the use of \"hallucination\" since I am unable to propose a better alternative. I did not list it as a weakness.\n\nI find the discussion on what the network learns and on evaluation metrics acceptable. The former may be hard to decipher, and the metrics are presented clearly for readers to draw their own conclusions.\n\nNitpicking: an affine transformation matrix does not encode image boundaries either. This may be a point where we are not on the same page.",
            " We would like to thank reviewer BNXs for his/her comments. Please find our answers below:\n\n- _(1) Authors admitted that the proposed method can only produce low resolution correspondence_: We would like to highlight that it is not the method itself that limits the resolution, but the implementation choices we had to make to train a network on 16GB memory GPUs in a reasonable amount of time (less than 24 hours in our case). By relaxing some of these constraints, there exists simple ways to significantly increase the resolution, such as gradient checkpointing (reduces the memory footprint but increases the training time) or using GPUs with larger memory.\n  \n- _(2) Is there any explanation why correspondence inpainting is harder for inpainting?_: We believe the inpainted correspondence maps are less peaked than the outpainted correspondence maps because outpainting a correspondent essentially consists in transferring the features from location $p_{s,i}$ in the source features to location $p_{t,i}$ in the target features to obtain $d_{s,i} = D_{T,pad} (p_{t,i})$ (please refer to figure 2 to understand these notations). In the inpainting case, $p_{s,i}$ is occluded by an object in the target image, and this object is (often) also visible in the source image at location $p_{s,occ}$ . Thus, to produce peaked correspondence maps for both $p_s$ and $p_{s,occ}$ , the network has to output features such that $d_{s,i} = D_{T,pad} (p_{t,i}) = d_{s,occ} $ which is more difficult than just $d_{s,i} = D_{T,pad} (p_{t,i})$. We added this paragraph in the updated version of the Appendix (Section A.2).\n  \n- _(3) Did authors define how to compute Kc for transforming points from image plane to correspondence plane? It seems that I am not able to find it in the main manuscript._: We left out that computational detail from the manuscript. $K_c$ simply corresponds to a rescaled and offsetted transformation of the original calibration matrix, found by changing the focal length and offsetting the principal point of the camera. We added the analytical formula to compute $K_c$ in the Appendix (Section C.1).\n\n- _(4) It would be better if authors could provide more analysis and ablation study on the relationship between accuracy of camera pose estimation and distribution of hallucinated correspondence. And in which case, hallucinated correspondence will be more accurate or less accurate._: We would like to highlight that we performed an analysis in that direction in section B.2 where we report the percentage of camera poses being correctly estimated for several values of $\\gamma$, which demonstrates the benefits of outpainting with a large $\\gamma$ for camera pose estimation. In Figure 6 we also showed that learning to inpaint does not bring any significant improvement. Outpainting improves the camera pose because outpainted correspondences are outside the field of view and thus complement the identified correspondences, and thus better constrain the camera pose estimate. On the contrary, inpainted correspondences are usually surrounded by identified correspondences, thus the information they provide is redundant and does not allow to better constrain the camera pose estimate. We added this analysis in the Appendix (Section B.3).\n\n- _(5) Authors should also cite several other related work for feature matching_: Thank you for pointing out these missing references, they have been added to the updated version of the paper.\n\nWe took into account the suggestions from all three reviewers and updated our submission accordingly.\nWe also uploaded a video demonstrating the generalization performances of NeurHal on both indoor and outdoor smartphone-captured videos in the supplementary material.",
            " We would like to thank reviewer nobf for his/her comments. Please find our answers below:\n\n- _The proposed method has limited technique contribution. For example, the original NRE [Germain 2021] is able to predict outpainting correspondences (it gives an extra category for un-matched keypoints). As a result, this paper adapts [Germain 2021] to solve this problem_: We would like to highlight that [Germain 2021] is **not** able to predict outpainting correspondences. They do consider an extra category for un-matched keypoints but the probability for that category is set to **zero**, ie. the network does not output any score for that category (it is stated in section 3 of [Germain 2021]: \"By definition, $C_{Q,n} (p_n^Q = out) := 0.$\"). Setting this probability to zero is due to the fact that [Germain 2021] considers a classical siamese CNN architecture that does not allow the features of both images to communicate. [Germain 2021] is what we called, in the introduction, a \"pure pattern recognition approach\". Moreover, even if [Germain 2021] were using a non-siamese architecture, their method would output a single score for the category \"un-matched keypoint\" which would allow the network to detect when the correspondent is not visible but would not be sufficient to outpaint the **location** of the correspondent.\n\n- _In figure 3, the baseline (uniform correspondence map) is too weak_:  Figure 3 illustrates the difference in NRE distributions for different kinds of hallucinated correspondences. In this plot the uniform distribution label is not a baseline, but merely an indicator which serves as a reference point to interpret the NRE values and argmax distances. Comparison against state-of-the-art methods (baselines) can be seen in Figure 4. \n\n- _In figure 4, I do not find the precision for identified keypoints. It is unfair for the other methods._: As stated in the section \"contributions\", our paper seeks to answer to the question: \"Can we derive a network architecture able to learn to hallucinate correspondences?\". Thus in Figure 4, we evaluated the performances on inpainting and outpainting to highlight that sota methods are not able to hallucinate correspondences while NeurHal can. We already know that all these methods are very good at identifying correspondences. In Figure 3, we did compute the identification histograms of NeurHal to highlight/compare the difficulty of inpainting and outpainting with respect to the task of identification.\n\n- _In camera pose estimation (section 4.2), the description of the baseline (correspondent to the first (light blue) method in figure 6) is not clear to me._: The baseline of Figure 6 corresponds to training our model using only identifiable (ie. covisible) keypoint correspondences. The other curves incorporate additional training data coming from non-covisible correspondences (inpainting and outpainting). We have made this clearer in the updated version of the paper.\n\n- _So, it is hard to tell if the proposed method is able to improve the state-of-the-art camera pose estimation methods._: Our experiments on absolute camera pose estimation demonstrate our method (trained jointly on identified, inpainted and outpainted correspondences) is indeed able to significantly improve the state-of-the-art, in the case of low-overlap image pairs (please see Figure 7, 12 and 13). Could you please specify why Figure 7, 12 and 13 do not clearly demonstrate the improvement of NeurHal over the state-of-the-art camera pose estimation methods?\n\n- _[...] improve the baselines, and compare with the state-of-the-art camera pose estimation methods._: To the best of our knowledge we did compare against the state-of-the-art camera pose estimation methods (LoFTR, DRCNet, SP+SG, R2D2, S2D). These are the best performing methods on visual localization benchmarks (e.g. [visuallocalization.net](https://www.visuallocalization.net/)).\n\nWe took into account the suggestions from all three reviewers and updated our submission accordingly.\nWe also uploaded a video demonstrating the generalization performances of NeurHal on both indoor and outdoor smartphone-captured videos in the supplementary material.",
            " We would like to thank reviewer uCMw for his/her comments. Please find our answers below:\n\n- _It is unclear to me what the network actually learns. I speculate that it learns to warp the source image to the target given the relative pose between the two cameras and the depth of the keypoint in the source image. The output of this process is a correspondence map, which suggests that multiple warpings are considered. This is not described clearly enough._: We agree with this analysis. However as you correctly pointed out, this is a \"speculation\" as there is no proof the network is actually doing this, so at the time of submission we decided not write anything about it. As requested, we updated our submission and added a paragraph in the Appendix (Sec A.2).\n\n- _The metrics used for evaluation are somewhat arbitrary, or not sufficiently justified. [...]  Camera pose estimation is considered correct when the rotation error is under 20 degrees and the translation error is under 1.5 m. The latter is uninformative without knowing the magnitude of the translation between the two cameras_: We agree that the choice of our thresholds are arbitrary, in fact the choice of thresholds in visual localization benchmarks is rarely backed by statistical evidence. In order to show that our thresholds are not too loose,  we reported in Figure 13 the performance of NeurHal, state-of-the-art feature matching methods and the identity pose, on ScanNet for several rotation and translation thresholds. We can see that 1.5m is not an \"easy\" threshold because the identity pose obtains poor performances. We added this analysis in the updated version of the Appendix (C.3) and refered to it in the main paper.\n\n- _Figure 3, for example, compares the results to random chance, which is a very weak baseline_: In this plot the uniform distribution label is not a baseline, but merely an indicator which serves as a reference point to interpret the NRE values and argmax distances. Comparison against state-of-the-art methods (baselines) can be seen in Figure 4. \n\n- _I find the term hallucination in the title marginally acceptable [...] Humans do not hallucinate, they predict or estimate._: We employed the term \"hallucinating correspondences\" as it is a significantly different task from \"identifying correspondences\". We believe the terms \"predicting correspondences\" or \"estimating correspondences\" embed the identification and hallucination tasks. As requested we either removed some instances of the term \"hallucination\" in the text, or appended the word \"predict\" in the updated version of the paper.\n\n- _Footnote 1 on p. 3 is unnecessary_: We replaced $K$ by $K_S$ and $K_T$ in Sec. 3.2 and removed the footnote.\n\n- _Section 3.2: the calibration matrix K_c does not encode the boundaries of the image_: We agree the term \"calibration matrix\" is misleading, we changed it to \"affine transformation matrix\".\n\n- _Section A.2 that discusses additional related work should be moved to the main paper_: We moved the additional related work from the Appendix to the main paper in its updated version.\n\n- _Figure 8: precision on the y-axis is perplexing. The description suggests that this should be recall._: We removed the term precision from the paper.\n\nWe took into account the suggestions from all three reviewers and updated our submission accordingly.\nWe also uploaded a video demonstrating the generalization performances of NeurHal on both indoor and outdoor smartphone-captured videos in the supplementary material.",
            "The paper proposes a method that accepts two partially overlapping images and a keypoint in one of them, and detects the location of the keypoint in the other image, regardless of whether it is visible, occluded, or outside the frame of the image. This is an interesting re-formulation of the correspondence estimation problem, the conventional formulation of which considers only keypoints that are visible in both images. Under the previous formulation, correspondence estimators are considered successful if they can declare that the correspondent of the keypoint is invisible in the target image. In addition to the capability to predict the location of invisible keypoints in isolation, the paper demonstrates that this capability is beneficial to camera pose estimation. *Strengths*\n\nThe capability to predict the location of points that have just become occluded or exited the frame is useful in many tasks, such as perception for robots where such predictions could guide the robot\u2019s next moves. Humans are good at this, but computer vision research has focused on the limited setting when the keypoints are visible in both images. Loss functions used for training networks for correspondence estimation only consider descriptors of visible keypoints. This is an important contribution of the paper and it goes beyond more qualitative work of predicting unseen objects based on context.\n\nThe approach builds upon the concept of Neural Reprojection Error (NRE) which was recently introduced by Germain et al. (2021). NRE does not require that the appearance of two image regions, that are hypothesized to be projections of the same 3D point, is similar. It can be viewed as an extension of the purely geometric reprojection error. Germain et al. did not consider occluded or out-of-bounds keypoints.\n\nI consider the unified treatment of visible, occluded and out-of-bounds keypoints an advantage.\n\nExperiments are thorough, findings are explained clearly, and limitations of the proposed method are pointed out. I consider the use of four publicly available datasets, two indoor and two outdoor, sufficient. Over one million keypoints are used in the experiments.\n\nGeneralization on additional datasets not used for training is also shown. This is important since it allows broader deployment of the algorithm.\n\nThe main limitation of NeurHal is low accuracy under favorable conditions, due to the low resolution of its output. (Improving this is left as a direction for future work.) \n\nThe appendices contain useful additional information, experiments and implementation details. \n\n*Weaknesses*\n\nIt is unclear to me what the network actually learns. I speculate that it learns to warp the source image to the target given the relative pose between the two cameras and the depth of the keypoint in the source image. The output of this process is a correspondence map, which suggests that multiple warpings are considered. This is not described clearly enough. (The fact that the visibility of keypoint in the target images does not need to be labeled is clear.)\n\nThe metrics used for evaluation are somewhat arbitrary, or not sufficiently justified. Figure 3, for example, compares the results to random chance, which is a very weak baseline. Camera pose estimation is considered correct when the rotation error is under 20 degrees and the translation error is under 1.5 m. The latter is uninformative without knowing the magnitude of the translation between the two cameras. 1.5 m may be a very small or a very large error depending on the input data.\n\n\n*Other Comments*\n\nI find the term hallucination in the title marginally acceptable, but I think that it is abused in the rest of the paper. For example in: \u201cLocal feature matching methods are only able to identify the correspondent\u2019s location when it is visible, while humans can also hallucinate its location when it is occluded or outside the field of view through geometric reasoning.\u201d Humans do not hallucinate, they predict or estimate. \n\nFootnote 1 on p. 3 is unnecessary. The equation it refers to holds for any images in general configuration sharing the same intrinsic parameters (K matrix). The latter constraint can be easily relaxed.\n\nSection 3.2: the calibration matrix K_c does not encode the boundaries of the image. Instead of a different matrix, what is needed are different boundary conditions for considering pixels to be within the image or not.\n\nSection A.2 that discusses additional related work should be moved to the main paper. It is very relevant to the problem at hand.\n\nFigure 8: precision on the y-axis is perplexing. The description suggests that this should be recall.\n\n The paper contains an important contribution as discussed above. The fact that predicting the locations of invisible points improves camera pose estimation provides further support that the method is useful in downstream tasks. ",
            "The paper proposed a deep method (NeurHal) to predict visible, occluded or out-of-view keypoint matching from source images to target images. In training, a correspondence map is obtained from ground truth of camera matrix/pose, and images. In testing, the model directly outputs three categories of matchings: identified, outputpainting and inpainting. As an application of NeurHal, the method is applied to camera pose estimation and tested on ScanNet and Megadepth. The experiments shows the method improves the estimation accuracy, particularly the outputpainting correspondences.  Strengths:\n+The paper proposed an new problem: if and how neural network can predict unseen scene (keypoints) from a source-target image pair. Human is able to heuristically guess the location of outpainting correspondences from the geometry. For example, the keypoint displacement is relative camera pose and scene depth. This paper uses a deep model to accomplish this goal.\n+ The paper proposed a method that is based on correspondence map and neural reprojection error [Germain 2021] with sufficient details.\n+ NeurHal is tested on public datasets for precision of correspondences and camera pose estimation, showing better result than baselines.\n\nWeakness:\n- The proposed method has limited technique contribution. For example, the original NRE [Germain 2021]  is able to predict outpainting correspondences (it gives an extra category for un-matched keypoints). As a result, this paper adapts [Germain 2021] to solve this problem.\n- In figure 3, the baseline (uniform correspondence map) is too weak. In figure 4, I do not find the precision for identified keypoints. It is unfair for the other methods.\n- In camera pose estimation (section 4.2), the description of the baseline (correspondent to the first (light blue) method in figure 6) is not clear to me. Because some methods highly rely on good correspondences but others are not. So, it is hard to tell if the proposed method is able to improve the state-of-the-art camera pose estimation methods.    I tend to reject the paper because of the lack of contribution in technique and insufficient experiments. I would like to see \n1. the technique differences, compared with [Germain 2021] .\n2. improve the baselines, and compare with the state-of-the-art camera pose estimation methods.",
            "In this manuscript, authors proposed a new problem of correspondence hallucination, in which for keypoint in the source image, its correspondence should be detected regardless if it is occluded or outside the field of view. In particular, the authors proposed a new model and training paradigm that learn to hallucinate correspondence by predicting its probability distribution of its location. Extensive experiments on both indoor and outdoor benchmarks demonstrated that proposed method  can help camera pose estimation and outperforms prior state of the art feature matching approaches.\n Strengths:\n(1) This work provides an interesting research direction correspondence learning, in which occluded and out of FoV correspondence can also help large-baseline camera pose estimation.\n(2) Based on this insight: authors proposed a new model for hallucinating the correspondence based on attention blocks.\n(3) Extensive experiments on indoor and outdoor benchmarks demonstrated that proposed model is able to hallucinate correspondence and outperform prior SoTA on wide baseline setting in terms of camera poes estimation.\n(4) Author provided detailed supplementary material and trained model and source code for better supporting reproducibility.\n\nWeakness:\n(1) Authors admitted that the proposed method can only produce low resolution correspondence, which can have a negative impact on localization accuracy. \n(2) Is there any explanation why correspondence inpainting is harder for inpainting?\n(3) Did authors define how to compute Kc for transforming points from image plane to correspondence plane? It seems that I am not able to find it in the main manuscript.\n(4) It would be better if authors could provide more analysis and ablation study on the relationship between accuracy of camera pose estimation and distribution of hallucinated correspondence. And in which case, hallucinated correspondence will be more accurate or less accurate.\n(5) Authors should also cite several other related work for feature matching:\nLearning feature descriptors using camera pose supervision, ECCV 2020\nSelf-Supervised Geometric Perception, CVPR 2021\nPatch2Pix: Epipolar-Guided Pixel-Level Correspondences, CVPR 2021\n In summary, based on the strength and weakness I mentioned, I like this paper's idea for hallucinating correspondence so that we can obtain better accuracy for camera pose estimation, although it would be better to have more analysis and ablation study for the impact of correspondence hallucination on camera pose estimation accuracy."
        ],
        "review_ratings": [
            -1,
            -1,
            -1,
            -1,
            -1,
            -1,
            -1,
            -1,
            -1,
            8,
            5,
            8
        ],
        "review_confidences": [
            -1,
            -1,
            -1,
            -1,
            -1,
            -1,
            -1,
            -1,
            -1,
            4,
            4,
            3
        ],
        "review_reply_tos": [
            "9MQG_FP3eQB",
            "Lv3RrS-8N1T",
            "F5nhRjo6xc",
            "dO24FT8L9rz",
            "n8UMfwiyzdr",
            "ag7x_m7iFKH",
            "VjkTIgQhQr8",
            "tz6cced5D1B",
            "y1QImMpCSy",
            "iclr_2022_jaLDP8Hp_gc",
            "iclr_2022_jaLDP8Hp_gc",
            "iclr_2022_jaLDP8Hp_gc"
        ],
        "critique_points": [
            {
                "Methodology": [],
                "Experiments": [],
                "Clarity": [
                    "The updated versions is better for illustration of concerns I pointed out."
                ],
                "Significance": [
                    "The proposed method is able to predict the location of correspondences for improving camera localization quality"
                ],
                "Novelty": [
                    "The technical contribution is sufficient"
                ]
            },
            {
                "Methodology": [
                    "Enhance the baseline with a geometric perspective using homography estimation and keypoint mapping, considering assumptions about scene depth variance and keypoint availability."
                ],
                "Experiments": [
                    "Show results for identified keypoints in Figure 4, focusing on matching accuracy for applications like 3D reconstruction.",
                    "Clarify if the task performed is the same as in https://www.visuallocalization.net/benchmark/, specifically regarding \"absolute-pose\" which appears to be relative pose.",
                    "Justify the use of different thresholds in Figure 7 (1.5m and 20\n^o) compared to the score board ((0.25m, 2\n^o) / (0.5m, 5\n^o) / (5m, 10\n^o)), especially the larger angular error threshold."
                ],
                "Clarity": [
                    "Highlight that NeurHal can output \"location\" of outpaint.",
                    "Clarify experiment settings."
                ],
                "Significance": [],
                "Novelty": []
            },
            {
                "Methodology": [],
                "Experiments": [],
                "Clarity": [
                    "The metrics are presented clearly for readers to draw their own conclusions.",
                    "The discussion on what the network learns is acceptable but may be hard to decipher."
                ],
                "Significance": [],
                "Novelty": []
            },
            {
                "Methodology": [
                    "It is unclear what the network actually learns. The description of the warping process and the correspondence map is not clear enough.",
                    "The metrics used for evaluation are somewhat arbitrary or not sufficiently justified. For example, comparing results to random chance is a weak baseline. The translation error threshold is uninformative without knowing the magnitude of the translation between the cameras."
                ],
                "Experiments": [
                    "Experiments are thorough. The use of four publicly available datasets is sufficient. Generalization on additional datasets not used for training is shown. However, low accuracy under favorable conditions due to the low resolution of the output is a limitation."
                ],
                "Clarity": [
                    "The term \"hallucination\" is abused. Humans predict or estimate, they do not hallucinate. Footnote 1 on p. 3 is unnecessary. Section 3.2: the calibration matrix K_c does not encode the boundaries of the image, but the paper suggests it does. Section A.2 that discusses additional related work should be moved to the main paper. Figure 8: precision on the y-axis is perplexing and should be recall."
                ],
                "Significance": [
                    "The capability to predict the location of points that have just become occluded or exited the frame is useful in many tasks. The unified treatment of visible, occluded and out-of-bounds keypoints is an advantage. Predicting the locations of invisible points improves camera pose estimation."
                ],
                "Novelty": [
                    "This is an interesting re-formulation of the correspondence estimation problem. The paper's approach builds upon the concept of Neural Reprojection Error (NRE) and extends it to handle occluded or out-of-bounds keypoints. It goes beyond more qualitative work of predicting unseen objects based on context."
                ]
            },
            {
                "Methodology": [
                    "The method is based on correspondence map and neural reprojection error.",
                    "Limited technique contribution as the method adapts [Germain 2021]."
                ],
                "Experiments": [
                    "The baseline (uniform correspondence map) in figure 3 is too weak.",
                    "Precision for identified keypoints is missing in figure 4.",
                    "Description of the baseline in camera pose estimation (section 4.2) is unclear.",
                    "Experiments on camera pose estimation need to be compared with state-of-the-art methods."
                ],
                "Clarity": [
                    "Description of the baseline in camera pose estimation (section 4.2) is not clear."
                ],
                "Significance": [
                    "The paper proposes a new problem: predicting unseen scene keypoints from image pairs."
                ],
                "Novelty": [
                    "Limited technique contribution as the original NRE [Germain 2021] can also predict outpainting correspondences.",
                    "The paper adapts [Germain 2021] to solve this problem."
                ]
            },
            {
                "Methodology": [
                    "The paper introduces a new model and training paradigm for hallucinating correspondence by predicting the probability distribution of its location.",
                    "Authors should define how to compute Kc for transforming points from the image plane to the correspondence plane."
                ],
                "Experiments": [
                    "Extensive experiments on both indoor and outdoor benchmarks demonstrated that the proposed method outperforms prior state-of-the-art feature matching approaches."
                ],
                "Clarity": [
                    "The manuscript should provide a clear definition of how to compute Kc for transforming points from the image plane to the correspondence plane.",
                    "Is there any explanation why correspondence inpainting is harder for inpainting?"
                ],
                "Significance": [
                    "The work provides an interesting research direction in correspondence learning, where occluded and out-of-FoV correspondence can help with large-baseline camera pose estimation.",
                    "The proposed method can potentially lead to better accuracy for camera pose estimation."
                ],
                "Novelty": [
                    "The paper proposes a new problem of correspondence hallucination.",
                    "The authors proposed a new model for hallucinating the correspondence based on attention blocks."
                ]
            }
        ],
        "meta_review_critique": {
            "Methodology": [],
            "Experiments": [],
            "Clarity": [],
            "Significance": [],
            "Novelty": []
        }
    }

In [45]:
ans = combine_critiques(sample_paper["critique_points"])

In [46]:
ans

{'Methodology': 'Enhance the baseline with a geometric perspective using homography estimation and keypoint mapping, considering assumptions about scene depth variance and keypoint availability. It is unclear what the network actually learns. The description of the warping process and the correspondence map is not clear enough. The metrics used for evaluation are somewhat arbitrary or not sufficiently justified. For example, comparing results to random chance is a weak baseline. The translation error threshold is uninformative without knowing the magnitude of the translation between the cameras. The method is based on correspondence map and neural reprojection error. Limited technique contribution as the method adapts [Germain 2021]. The paper introduces a new model and training paradigm for hallucinating correspondence by predicting the probability distribution of its location. Authors should define how to compute Kc for transforming points from the image plane to the correspondence p

In [49]:
results = ai_agent_pipeline(sample_paper)

Step 1: General SoTA Search


> Entering new AgentExecutor chain...
I need to find the latest research related to the paper 'Visual Correspondence Hallucination'. I should start by searching for the paper on ArXiv, Google Scholar, and Semantic Scholar to find the original paper and any citations or related work. Then, I'll use Tavily to find more recent web results that might discuss the paper or its applications.
Action: ArxivSearch
Action Input: "Visual Correspondence Hallucination"
Observation: Published: 2024-05-06
Title: A Survey on Hallucination in Large Vision-Language Models
Authors: Hanchao Liu, Wenyuan Xue, Yifei Chen, Dapeng Chen, Xiutian Zhao, Ke Wang, Liping Hou, Rongjun Li, Wei Peng
Summary: Recent development of Large Vision-Language Models (LVLMs) has attracted
growing attention within the AI landscape for its practical implementation
potential. However, ``hallucination'', or more specifically, the misalignment
between factual visual content and corresponding textual ge

In [53]:
results["Retrieved Evidence"]

{'Methodology': '*   **Lack of clarity in network learning and warping process:** "SeFENet: Robust Deep Homography Estimation via Semantic-Driven Feature Enhancement"\n*   **Arbitrary or insufficiently justified metrics:** "A Large Scale Homography Benchmark"\n*   **Weak baseline comparison (random chance):** "Precise Aerial Image Matching based on Deep Homography Estimation" (indirectly, by comparing to conventional works)',
 'Clarity': 'I am unable to find specific research papers that directly support or contradict the critiques of the "Clarity" paper without more context.',
 'Experiments': 'Based on the search results and the identified papers, here\'s what I can say about the initial critiques:\n\n*   **Absolute vs. Relative Pose:** The papers confirm the importance of distinguishing between absolute and relative pose regression in visual localization. Some methods directly regress the absolute pose, while others estimate the relative pose between images. Combining both approaches

# Running pipeline on the whole dataset

In [26]:
import json
input_file = "../../data/processed/critique_points/critique_points_llm_2.json"
papers = json.load(open(input_file, "r"))

In [27]:
SAVE_PATH = "../../data/processed/search/search_intermediate_2.json"

In [28]:
import os

if os.path.exists(SAVE_PATH):
    with open(SAVE_PATH, "r") as f:
        processed_papers = json.load(f)
else:
    processed_papers = {}

In [29]:
for paper in papers: 
    paper = ai_agent_pipeline(paper)

✅ Skipping Quasi-Newton policy gradient algorithms, already fully processed.

✅ Skipping Multi-Domain Self-Supervised Learning, already fully processed.

✅ Skipping Hindsight Foresight Relabeling for Meta-Reinforcement Learning, already fully processed.

✅ Skipping Using Hindsight to Anchor Past Knowledge in Continual Learning, already fully processed.

✅ Skipping Predictive Coding beyond Gaussian Distributions, already fully processed.

✅ Skipping A Target-Agnostic Attack on Deep Models: Exploiting Security Vulnerabilities of Transfer Learning, already fully processed.

✅ Skipping A Near-Optimal Primal-Dual Method for Off-Policy Learning in CMDP, already fully processed.

✅ Skipping Enabling Deep Spiking Neural Networks with Hybrid Conversion and Spike Timing Dependent Backpropagation, already fully processed.

✅ Skipping Efficient Adversarial Training without Attacking: Worst-Case-Aware Robust Reinforcement Learning, already fully processed.

✅ Skipping Taking ROCKET on an efficiency

C:\Users\TIRATH BHATHAWALA\AppData\Local\Temp\ipykernel_21348\1805815892.py:9: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = agent.run(query)


The question asks for research papers related to critiques about specifying algorithms formally instead of text-based outlines, and the specificity of empirical results to Hex. I will start by searching for papers related to specifying algorithms formally using ArxivSearch.
Action: ArxivSearch
Action Input: "formal specification of algorithms vs text-based outlines"
Observation: Published: 2023-06-13
Title: A Markovian Formalism for Active Querying
Authors: Sid Ijju
Summary: Active learning algorithms have been an integral part of recent advances in
artificial intelligence. However, the research in the field is widely varying
and lacks an overall organizing leans. We outline a Markovian formalism for the
field of active learning and survey the literature to demonstrate the
organizing capability of our proposed formalism. Our formalism takes a
partially observable Markovian system approach to the active learning process
as a whole. We specifically outline how querying, dataset augmentat

In [ ]:
output_file = "../../data/processed/search/search_results_2.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(papers, f, indent=4)

In [31]:
output_papers = json.load(open(output_file, "r"))

In [32]:
len(output_papers)

100